### Trabalho realizado pelo grupo G18

- #### José Silva A100105
- #### Alexandra Calafate A100060


# TP4 - Problema 1:


a) Defina um autómato híbrido que descreva a dinâmica do sistema segundo as notas abaixo indicadas e com os “switchs” por si escolhidos. 
        Os “switchs” (“jumps”) são uma  componente de projeto deste trabalho; cabe ao aluno definir quais devem ser estas  condições de modo a que o sistema tenha um comportamento desejável: imobilize-se depressa e não “derrape” muito.

In [2]:
from z3 import *
from pysmt.shortcuts import *
import pysmt.typing
import itertools
import matplotlib.pyplot as plt
from math import ceil

In [4]:
timer = 3

vars = ['T','V','R','M','Timer']
def declare(S,i):
    s = {}
    s['T'] = Symbol('T'+S+str(i), REAL)
    s['V'] = Symbol('V'+S+str(i), REAL)
    s['R'] = Symbol('R'+S+str(i), REAL)
    s['M'] = Symbol('M'+S+str(i), INT)
    #s['c'] = Symbol('c'+S+str(i), REAL)
    s['Timer'] = Symbol('Timer'+S+str(i), REAL)
    return s

def init(s, vi):
    return And(Equals(s['T'], Real(0)), Equals(s['V'], Real(vi)), Equals(s['R'], Real(vi)), Equals(s['M'], Int(0)))




In [ ]:
disc = []
i = 0
while i<=a2*P*tau:
    disc.append(i)
    i += 0.5
    
dt = 0.01
    
def trans(s, p):
    
    #untimed
    startToFree = And(Equals(s['M'],Int(0)), Equals(p['M'],Int(1)), Equals(s['T'],p['T']), Equals(s['V'],p['V']), Equals(s['R'],p['R']),
                         Equals(p['Timer'], Real(0)))
    
    stoppingToBlocked = And(Equals(s['M'],Int(2)), Equals(p['M'],Int(3)), Equals(s['T'],p['T']), s['V']>0, s['R']>=0, 
                            Equals(s['V'],p['V']), Equals(s['R'],p['R']), Equals(p['Timer'],Real(0)), s['V']-s['R']<e)
    
    blockedToFree = And(Equals(s['M'],Int(3)), Equals(p['M'],Int(1)) ,Equals(s['T'],p['T']),s['V']>=0,s['R']>=0, 
                       Equals(s['V'],p['V']), Equals(s['R'],p['R']), s['Timer']>=tau, 
                       Equals(p['Timer'],Real(0)))
    
    
    freeToStopping  =  And(Equals(s['M'],Int(1)), Equals(p['M'],Int(2)) ,Equals(s['T'],p['T']), s['V']>=0,s['R']>=0, 
                          Equals(s['V'],p['V']), Equals(s['R'],p['R']), s['Timer']>=tau)
    
    
    stoppingToStopped = And(Equals(s['M'],Int(2)), Equals(p['M'],Int(4)) ,Equals(s['T'],p['T']),
                           s['V']<e, s['R']<e, Equals(p['V'], Real(0)), Equals(p['R'], Real(0)))
    
    stoppedToStopped = And(Equals(s['M'],Int(4)), Equals(p['M'],Int(4)), Equals(s['T'] , p['T']), 
                          Equals(s['V'],p['V']), Equals(s['R'], p['R']))
    
    #timed
    freeToFree = Or([And(Equals(s['M'],Int(1)),Equals(p['M'],Int(1)),p['T'] - s['T'] > dt, s['V']>=0,s['R']>=0,
                        p['V']>=0, p['R']>=0, 
                        p['Timer']<=tau,Equals(p['Timer'],s['Timer']+p['T']-s['T']), 
                        s['V']-s['R']<b+0.5, s['V']-s['R']>=b-0.5, 
                        Equals(p['V'],(s['V']+(-c2*b)*(p['T']-s['T']))),
                        Equals(p['R'],(s['R']+(-a2*P + c2*b)*(p['T']-s['T']))))for b in disc])
    
    stoppingToStopping = Or([And(Equals(s['M'],Int(2)),Equals(p['M'],Int(2)),p['T'] - s['T'] > dt,
                        s['V']-s['R']>=e, p['V']-p['R']>=0,   
                        s['V']>=0,s['R']>=0, p['V']>=0, p['R']>=0,
                        s['V']-s['R']<b+0.5, s['V']-s['R']>=b-0.5, 
                        Equals(p['V'],(s['V']+(-c1*b)*(p['T']-s['T']))),
                        Equals(p['R'],(s['R']+(-a2*P + c1*b)*(p['T']-s['T']))))for b in disc])
    
    blockedToBlocked = And(Equals(s['M'],Int(3)),Equals(p['M'],Int(3)),p['T'] - s['T']> dt,s['V']>=0,s['R']>=0,
                          p['V']>=0, p['R']>=0, 
                          p['Timer']<=tau,Equals(p['Timer'],s['Timer']+p['T']-s['T']),
                          Equals(p['V'], p['R']), 
                          Equals(p['R'], s['R'] + (-a1*P)*(p['T']-s['T']))) 
    
    return Or(startToFree, stoppingToBlocked, blockedToFree, freeToStopping, stoppingToStopped, stoppedToStopped,
              freeToFree, stoppingToStopping, blockedToBlocked)

In [ ]:
def genTrace(vars,init,trans,n):
    with Solver(name="z3") as solver:
        X = [declare('X',i) for i in range(n+1)]   # cria n+1 estados (com etiqueta X)
        I = init(X[0],vi)
        Tks = [ trans(X[i],X[i+1]) for i in range(n) ]
        if solver.solve([I,And(Tks)]):      # testa se I /\ T^n  é satisfazível
            for i in range(n+1):
                print("Estado:",i)
                for v in X[i]:
                    print("          ",v,'=',float(solver.get_py_value(X[i][v])))
        else:
            print("Help!")

In [ ]:
genTrace(vars, init, trans, 20)

In [ ]:
def genTraceEnd(vars,init,trans,n):
    with Solver(name="z3") as solver:
        X = [declare('X',i) for i in range(n+1)]   # cria n+1 estados (com etiqueta X)
        I = init(X[0],vi)
        Tks = [ trans(X[i],X[i+1]) for i in range(n) ]
        End = Equals(X[-1]['M'], Int(4))
        if solver.solve([I,And(Tks),End]):      # testa se I /\ T^n  é satisfazível
            for i in range(n+1):
                print("Estado:",i)
                for v in X[i]:
                    print("          ",v,'=',float(solver.get_py_value(X[i][v])))
        else:
            print("A execução não termina em", n, "passos.")

In [ ]:
genTraceEnd(vars, init, trans, 15)

In [ ]:
def bmc_always(declare,init,trans,inv,end,time,K):
    for k in range(1,K+1):
        with Solver(name="z3") as solver:
            trace = [declare('X',i) for i in range(k)]
            solver.add_assertion(init(trace[0], vi))
            for i in range(k-1):
                solver.add_assertion(trans(trace[i],trace[i+1]))
                solver.add_assertion(Not(inv(trace[i],trace[i+1])))
            solver.add_assertion(Not(end(trace[-1],time)))
            if solver.solve():
                print(f"As propriedades não são validas para o seguinte traço de tamanho <= %d" % k)
                i = 0
                for traco in trace:
                    print("Estado:",i)
                    i+=1
                    for v in traco:
                        print("          ",v,'=',float(solver.get_py_value(traco[v])))
                #return
    print(f"Propriedades válida para traços de tamanho <= %d." % k)

In [ ]:
def testStop(state,time):
    return Implies(state['T']>=Real(time), Or(Equals(state['M'],Int(4)),And(state['V']<=Real(0),state['R']<=Real(0))))

def inv(pre,pos):
    return Implies(pre['T']<pos['T'],pre['V']>pos['V'])

In [ ]:
bmc_always(declare,init,trans,inv,testStop,4,15)

In [ ]:
bmc_always(declare,init,trans,inv,testStop,1,15)